In [ ]:
import numpy as np
import cv2

In [ ]:
class Camera:
    def __init__(self):
        pass
    
    
    def project_points(self):
        pass


class Environment:
    def __init__(self):
        #set points' coords
        pass
    
    def get_observation(self):
        """
        returns points' coordinates on camera image
        @return array of points': [(i1, j1), (i2, j2), ... (i6, j6)]
        """
        pass
    
    def running_cost(self, projected_points_coords):
        """
        returns sum of distances from true points to target values
        @return float distance
        """
        assert all([len(point) for point in projected_points_coords] == 2), "you should pass points' pixel coords, not (x, y, z)!"
        pass
    
class Agent: #camera
    def __init__(self):
        self.coords = np.zeros(3, dtype=np.int32)
        pass

    def move(self, delta):
        """
        adds delta to current camera coordinates
        """
        pass
    
    

In [ ]:
array([[-138.,   28.,  -46.],
       [-138.,   43.,  -45.],
       [-149.,   19.,  -45.],
       [-149.,   51.,  -44.],
       [-163.,   28.,  -45.],
       [-163.,   43.,  -44.]])